<a href="https://colab.research.google.com/github/PriyaMurali/Data-Analysis/blob/main/Spark_Capstone_Project_Credit_Card_Approval_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
!apt-get update
!apt-get install --fix-missing openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,937 kB]
Hit:14 http://ppa.l

In [2]:
!pip install mysql-connector-python
import findspark
findspark.init()
from pyspark.sql import SparkSession

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 1.6 MB/s 


In [3]:
spark = SparkSession \
        .builder \
        .appName("SparkCapstoneProject") \
        .getOrCreate()

In [5]:
from pyspark.sql.types import *
customer_schema = StructType([
StructField("CustomerName", StringType(), True),
StructField("DOB", StringType(), True),
StructField("SSN", StringType(), True),
StructField("City", StringType(), True),
StructField("State", StringType(), True),
StructField("ZipCode", LongType(), True),
StructField("ExistingProducts", StringType(), True),
StructField("OtherBankProducts1", StringType(), True),
StructField("CreditSpent1", LongType(), True),
StructField("OtherBankProducts2", StringType(), True),
StructField("CreditSpent2", LongType(), True),
StructField("OtherBankProducts3", StringType(), True),
StructField("CreditSpent3", LongType(), True),
StructField("DefaulterFlag", StringType(), True)
])
CustomerDS = spark.read.csv(path='/content/CustomersReferenceDataset.csv',sep=',',schema=customer_schema)
CustomerDS.show()

+------------+----------+-------+-------+-----------+-------+----------------+--------------------+------------+--------------------+------------+------------------+------------+-------------+
|CustomerName|       DOB|    SSN|   City|      State|ZipCode|ExistingProducts|  OtherBankProducts1|CreditSpent1|  OtherBankProducts2|CreditSpent2|OtherBankProducts3|CreditSpent3|DefaulterFlag|
+------------+----------+-------+-------+-----------+-------+----------------+--------------------+------------+--------------------+------------+------------------+------------+-------------+
|         Bob|23-08-1981|SSN0001|Atlanta|    Georgia|  30301|      SavingsACC| FCIBankPlatinumCard|         250|   HSBBankCreditCard|         450|  HSBBankCoralCard|         250|            N|
|        Jack|17-09-1984|SSN0002|Chicago|   Illinois|  60601|      CurrentACC|NPCCBankPlatinumCard|        1250|   HSBBankCreditCard|          50|  HSBBankCoralCard|         250|            N|
|       Krish|17-03-1983|SSN0003| B

In [6]:
from pyspark.sql.types import *
cc_schema = StructType([
StructField("CustomerName", StringType(), True),
StructField("DOB", StringType(), True),
StructField("SSN", StringType(), True),
StructField("MailID", StringType(), True),
StructField("PhoneNumber", LongType(), True),
StructField("City", StringType(), True),
StructField("State", StringType(), True),
StructField("ZipCode", LongType(), True),
StructField("CreditLimit", LongType(), True)
])
CcDS = spark.read.csv(path='/content/CreditCardApplicationData.csv',sep=',',schema=cc_schema)
CcDS.show()

+------------+----------+-------+----------------+-----------+-------+-----------+-------+-----------+
|CustomerName|       DOB|    SSN|          MailID|PhoneNumber|   City|      State|ZipCode|CreditLimit|
+------------+----------+-------+----------------+-----------+-------+-----------+-------+-----------+
|         Bob|23-08-1981|SSN0001|    Bob@mail.com| 1287655553|Atlanta|    Georgia|  30301|       3000|
|        Jack|17-09-1984|SSN0002|   Jack@mail.com| 9776543221|Chicago|   Illinois|  60601|       2500|
|       Krish|17-03-1983|SSN0003|  Krish@mail.com| 9987765234| Biloxi|Mississippo|  39530|       1500|
|       Jacob|22-02-1978|SSN0004|  Jacob@mail.com| 4335544553| Biloxi|Mississippo|  39530|       1500|
|       Ethen|21-08-1979|SSN0005|  Ethen@mail.com| 8888777722|Chicago|   Illinois|  60601|       5500|
|     William|21-06-1982|SSN0006|William@mail.com|  223377882|Atlanta|    Georgia|  30301|       1000|
|       Jayde|23-08-1986|SSN0007|  Jayde@mail.com|44557768443| Biloxi|Mis

In [7]:
CustomerDS.createOrReplaceTempView("CustomerData")
CcDS.createOrReplaceTempView("CreditCardData")

In [8]:
spark.sql("show tables").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |creditcarddata|       true|
|         |  customerdata|       true|
+---------+--------------+-----------+



In [10]:
#Verify whether customer is defaulter or not.
joinedDf=spark.sql("select CustomerData.CustomerName,CustomerData.SSN,(CreditSpent1+CreditSpent2+CreditSpent3) as TotalCreditSpent,CreditLimit,DefaulterFlag \
          from CustomerData join CreditCardData \
          on CustomerData.SSN = CreditCardData.SSN")
joinedDf.show()

+------------+-------+----------------+-----------+-------------+
|CustomerName|    SSN|TotalCreditSpent|CreditLimit|DefaulterFlag|
+------------+-------+----------------+-----------+-------------+
|         Bob|SSN0001|             950|       3000|            N|
|        Jack|SSN0002|            1550|       2500|            N|
|       Krish|SSN0003|             750|       1500|            N|
|       Jacob|SSN0004|             750|       1500|            N|
|       Ethen|SSN0005|            1980|       5500|            Y|
|     William|SSN0006|             950|       1000|            N|
|       Jayde|SSN0007|            1080|       1000|            N|
|       Ethan|SSN0008|             850|       4500|            Y|
|      Jayden|SSN0009|            1040|       2000|            N|
|      Anthon|SSN0010|            1954|        500|            Y|
|      Daniel|SSN0011|             924|        500|            N|
|       Janes|SSN0012|            1824|       3500|            N|
|        J

In [26]:
from pyspark.sql.functions import when,col,lit
joinedDf = joinedDf.withColumn("CreditWithinLimit", when(col("CreditLimit") > col("TotalCreditSpent"),lit("Y")) \
.otherwise(lit("N")))
joinedDf.show()

+------------+-------+----------------+-----------+-------------+-----------------+
|CustomerName|    SSN|TotalCreditSpent|CreditLimit|DefaulterFlag|CreditWithinLimit|
+------------+-------+----------------+-----------+-------------+-----------------+
|         Bob|SSN0001|             950|       3000|            N|                Y|
|        Jack|SSN0002|            1550|       2500|            N|                Y|
|       Krish|SSN0003|             750|       1500|            N|                Y|
|       Jacob|SSN0004|             750|       1500|            N|                Y|
|       Ethen|SSN0005|            1980|       5500|            Y|                Y|
|     William|SSN0006|             950|       1000|            N|                Y|
|       Jayde|SSN0007|            1080|       1000|            N|                N|
|       Ethan|SSN0008|             850|       4500|            Y|                Y|
|      Jayden|SSN0009|            1040|       2000|            N|           

In [28]:
from pyspark.sql.functions import when,col,lit
StatusDF = joinedDf.withColumn("Status",when((col('DefaulterFlag') =='N') & (col('CreditWithinLimit') == 'Y'), lit("Approved")) \
                   .otherwise(lit("Rejected"))) \
                   .select(col("CustomerName"),col("SSN"),col("Status"))
StatusDF.show()
#.select(col("CustomerName"),col("SSN"),col("Status")) \


+------------+-------+--------+
|CustomerName|    SSN|  Status|
+------------+-------+--------+
|         Bob|SSN0001|Approved|
|        Jack|SSN0002|Approved|
|       Krish|SSN0003|Approved|
|       Jacob|SSN0004|Approved|
|       Ethen|SSN0005|Rejected|
|     William|SSN0006|Approved|
|       Jayde|SSN0007|Rejected|
|       Ethan|SSN0008|Rejected|
|      Jayden|SSN0009|Approved|
|      Anthon|SSN0010|Rejected|
|      Daniel|SSN0011|Rejected|
|       Janes|SSN0012|Approved|
|        John|SSN0013|Approved|
|        Jose|SSN0014|Approved|
|        Knox|SSN0015|Approved|
|        Enzo|SSN0016|Approved|
|      Karter|SSN0017|Approved|
|      Brooks|SSN0018|Approved|
|      Alfred|SSN0019|Approved|
|       Oscar|SSN0020|Rejected|
+------------+-------+--------+
only showing top 20 rows



In [30]:
StatusDF.write.csv("/content/CreditcardStatusReport.csv",header=True)
StatusDF.count()

100800

In [ ]:
spark.stop()